In [1]:
import autograd.numpy as np  
from autograd import grad, elementwise_grad
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

In [2]:
#Defining some activation functions and their derivative
def ReLU(z):
    return np.where(z > 0, z, 0)

def ReLU_der(z):
    return np.where(z > 0, 1, 0)

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sigmoid_der(z):
    return sigmoid(z) * (1 - sigmoid(z))

def mse(predict, target):
    return np.mean((predict - target) ** 2)

def mse_der(predict, target):
    return 2 * (predict - target) / np.prod(predict.shape)

def softmax(z):
    """Compute softmax values for each set of scores in the rows of the matrix z.
    Used with batched input data."""
    e_z = np.exp(z - np.max(z, axis=1)[:, np.newaxis]) #substract max per row, avoids instability
    return e_z / np.sum(e_z, axis=1)[:, np.newaxis]

def softmax_vec(z):
    """Compute softmax values for each set of scores in the vector z.
    Use this function when you use the activation function on one vector at a time"""
    e_z = np.exp(z - np.max(z))
    return e_z / np.sum(e_z)

def softmax_der(z):
    return np.ones_like(z) 
#Purely placeholder, combined derivative 
#Cross entropy + softmax simplifies to predict-target

def cross_entropy(predict, target):
    return np.mean(-np.sum(target * np.log(predict + 1e-10), axis=1))

def cross_entropy_der(predict, target):
    return (predict - target) / predict.shape[0]


In [3]:
#Initializing creation of n layers for batched input shapes
def create_layers_batch(network_input_size, layer_output_sizes):
    layers = []

    i_size = network_input_size
    for layer_output_size in layer_output_sizes:
        std = np.sqrt(2.0 / i_size)
        W = np.random.randn(layer_output_size, i_size) * std
        W = W.T
        b = np.random.randn(layer_output_size)
        layers.append((W, b))

        i_size = layer_output_size
    return layers

#Applying weights, bias and activation function and passing forward
def feed_forward_saver_batch(input, layers, activation_funcs):
    layer_inputs = []
    zs = []
    a = input
    for (W, b), activation_func in zip(layers, activation_funcs):
        layer_inputs.append(a)
        z = a @ W + b
        a = activation_func(z)

        zs.append(z)

    return layer_inputs, zs, a

#Same, but when saving layer_inputs and zs is not needed
def feed_forward(input, layers, activation_funcs):
    a = input
    for (W, b), activation_func in zip(layers, activation_funcs):
        z = a @ W + b
        a = activation_func(z)
    return a

#Verify gradients
def cost(input, layers, activation_funcs, target):
    predict = feed_forward_saver_batch(input, layers, activation_funcs)[2]
    return mse(predict, target)

#Computing gradients
def backpropagation_batch(
    input, layers, activation_funcs, target, activation_ders, cost_der=mse_der
):
    layer_inputs, zs, predict = feed_forward_saver_batch(input, layers, activation_funcs)

    layer_grads = [() for layer in layers]

    # We loop over the layers, from the last to the first
    for i in reversed(range(len(layers))):
        layer_input, z, activation_der = layer_inputs[i], zs[i], activation_ders[i]

        if i == len(layers) - 1:
            # For last layer we use cost derivative as dC_da(L) can be computed directly
            dC_da = cost_der(predict, target)
        else:
            # For other layers we build on previous z derivative, as dC_da(i) = dC_dz(i+1) * dz(i+1)_da(i)
            (W, b) = layers[i + 1]
            dC_da = dC_dz @ W.T

        dC_dz = dC_da * activation_der(z)
        dC_dW = layer_input.T @ dC_dz  #W gradients for batches
        dC_db = np.sum(dC_dz, axis=0)   #sum bias gradients, batch dim

        layer_grads[i] = (dC_dW, dC_db)

    return layer_grads

In [4]:
#Gradient verification with autograd

network_input_size = 4
batch_size = 400
layer_output_sizes = [3, 4]
activation_funcs = [sigmoid, ReLU]
activation_ders = [sigmoid_der, ReLU_der]
la = create_layers_batch(network_input_size, layer_output_sizes)

x = np.random.randn(batch_size, network_input_size)
input = x
target = np.random.rand(4)

computed = backpropagation_batch(input, la, activation_funcs, target, activation_ders)
print(computed[-1][0][0])

cost_grad = grad(cost, 1)
autoG = cost_grad(input, la, activation_funcs, target)
print(autoG[-1][0][0])

diff = 0
for i in range(len(computed)):
    diff += abs(computed[-1][0][i][0])-abs(autoG[-1][0][0][0])
if diff <= 10**-6:
    print("Same gradients")


[-5.43607120e-02  0.00000000e+00 -5.65599844e-05  1.25980236e-01]
[-5.43607120e-02  0.00000000e+00 -5.65599844e-05  1.25980236e-01]
Same gradients


In [5]:
"""
#Applying FFNN base above to classification problem, MNIST:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

#Fetch the MNIST dataset
mnist = fetch_openml('mnist_784', version=1, as_frame=False, parser='auto')

#Extract data
X = mnist.data
y_labels = mnist.target.astype(int)
X = X / 255.0
 

#One-hot encode labels
y = np.eye(10)[y_labels] #Convert to integers

#Split into train and test
#y_train_labels and y_test_labels for scikit-learn, expects integer labels
X_train, X_test, y_train, y_test, y_train_labels, y_test_labels = train_test_split(
    X, y, y_labels, test_size=0.1, random_state=42
)

print(X_train.shape)
print(y_train.shape)
"""

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv"
data = pd.read_csv(url, sep=';')

X = data.drop('quality', axis=1).values
y_reg = data['quality'].values.astype(float)           # regression target
y_cls = data['quality'].values - 3                     # classes 0–6 for classification

# Optional: turn into binary classification (good ≥7, bad ≤5, neutral=6→bad or remove)
# y_binary = (data['quality'] >= 7).astype(int)

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, yreg_train, yreg_test = train_test_split(X, y_reg, test_size=0.2, random_state=42)
#X_train_cls, X_test_cls, ycls_train, ycls_test = train_test_split(X, y_cls, test_size=0.2, random_state=42)

print(y_reg)

[6. 6. 6. ... 6. 7. 6.]


In [6]:
def accuracy(predictions, targets):
    one_hot_predictions = np.zeros(predictions.shape)
    for i, prediction in enumerate(predictions):
        one_hot_predictions[i, np.argmax(prediction)] = 1
    return accuracy_score(one_hot_predictions, targets)

In [7]:
import numpy as np

# ============================
# OPTIMIZER FACTORY + STATE
# ============================

def create_optimizer(optimizer_name="adam", lr=0.001):
    if optimizer_name == "sgd":
        return {"name": "sgd", "lr": lr}
    
    elif optimizer_name == "adam":
        return {
            "name": "adam",
            "lr": lr,
            "beta1": 0.9,
            "beta2": 0.999,
            "epsilon": 1e-8,
            "t": 0,
            "m": [],   # will hold (m_W, m_b) for each layer
            "v": []    # will hold (v_W, v_b) for each layer
        }
    
    elif optimizer_name == "rmsprop":
        return {
            "name": "rmsprop",
            "lr": lr,
            "beta": 0.99,
            "epsilon": 1e-8,
            "v": []
        }
    else:
        raise ValueError("Unknown optimizer")

# ============================
# MAIN UPDATE FUNCTION
# ============================

def update_parameters(layers, grads, optimizer_config, t=None):
    """
    One function to rule them all.
    Pass in your layers, gradients, and optimizer config → returns updated layers
    """
    name = optimizer_config["name"]
    
    if name == "sgd":
        lr = optimizer_config["lr"]
        updated_layers = []
        for (W, b), (dW, db) in zip(layers, grads):
            W = W - lr * dW
            b = b - lr * db
            updated_layers.append((W, b))
        return updated_layers

    elif name == "adam":
        # Initialize m and v on first call
        if len(optimizer_config["m"]) == 0:
            optimizer_config["m"] = [(np.zeros_like(W), np.zeros_like(b)) for W, b in layers]
            optimizer_config["v"] = [(np.zeros_like(W), np.zeros_like(b)) for W, b in layers]
        
        beta1 = optimizer_config["beta1"]
        beta2 = optimizer_config["beta2"]
        lr = optimizer_config["lr"]
        eps = optimizer_config["epsilon"]
        t = optimizer_config["t"] + 1
        optimizer_config["t"] = t  # update timestep

        updated_layers = []
        for i, ((W, b), (dW, db)) in enumerate(zip(layers, grads)):
            m_W, m_b = optimizer_config["m"][i]
            v_W, v_b = optimizer_config["v"][i]

            # Adam update (per parameter)
            m_W = beta1 * m_W + (1 - beta1) * dW
            m_b = beta1 * m_b + (1 - beta1) * db
            v_W = beta2 * v_W + (1 - beta2) * (dW ** 2)
            v_b = beta2 * v_b + (1 - beta2) * (db ** 2)

            m_hat_W = m_W / (1 - beta1 ** t)
            m_hat_b = m_b / (1 - beta1 ** t)
            v_hat_W = v_W / (1 - beta2 ** t)
            v_hat_b = v_b / (1 - beta2 ** t)

            W = W - lr * m_hat_W / (np.sqrt(v_hat_W) + eps)
            b = b - lr * m_hat_b / (np.sqrt(v_hat_b) + eps)

            # Save back
            optimizer_config["m"][i] = (m_W, m_b)
            optimizer_config["v"][i] = (v_W, v_b)
            updated_layers.append((W, b))

        return updated_layers

    # You can easily add RMSprop, AdamW, etc. here

In [8]:
def dropout(x, keep_prob=0.8, training=True):
    """
    keep_prob = probability a neuron is KEPT (not dropped)
    0.8 → drop 20 % of neurons (best for hidden layers)
    0.5 → drop 50 % (sometimes used, but 0.8–0.9 is better for ReLU nets)
    """
    if training:
        mask = np.random.rand(*x.shape) < keep_prob     # random mask
        return x * mask / keep_prob                     # ← INVERTED DROPOUT (important!)
    else:
        return x                                        # no dropout at test time
    
def ReLU_with_dropout(z, keep_prob=0.8, training=True):
    a = ReLU(z)
    return dropout(a, keep_prob=keep_prob, training=training)

In [9]:

def train_network(X_train, y_train, layers, activation_funcs, activation_ders, learning_rate, epochs, batch_size=100):
    losses = []
    train_accs = []  
    test_accs = [] 
    #
    optimizer = create_optimizer("adam", lr=0.001)   # or "sgd", lr=0.05
    #
    for epoch in range(epochs):
        # Shuffle digits, avoids learning misleading patterns. Independent data
        perm = np.random.permutation(X_train.shape[0])
        X_shuffled = X_train[perm] 
        y_shuffled = y_train[perm]
        
        # Mini-batch loop
        for start in range(0, X_train.shape[0], batch_size):
            end = start + batch_size
            input_batch = X_shuffled[start:end]
            target_batch = y_shuffled[start:end]
            
            layers_grad = backpropagation_batch(input_batch, layers, activation_funcs, target_batch, activation_ders, cost_der=cross_entropy_der)
            #
            layers = update_parameters(layers, layers_grad, optimizer)
            #
            for (W, b), (W_g, b_g) in zip(layers, layers_grad):
                W -= learning_rate * W_g
                b -= learning_rate * b_g
        
        # Compute loss after epoch
        train_predictions = feed_forward(X_train, layers, activation_funcs)
        test_predictions = feed_forward(X_test, layers, activation_funcs)
        loss = cross_entropy(train_predictions, y_train)
        TESTacc = accuracy(test_predictions, y_test)
        TRAINacc = accuracy(train_predictions, y_train)
        losses.append(loss)
        train_accs.append(TRAINacc)  
        test_accs.append(TESTacc)    
        print(f"Epoch {epoch+1}, Loss: {loss:.3f}, Test Accuracy: {TESTacc:.3f}, Train Accuracy: {TRAINacc:.3f}") 
    
    return layers, losses, train_predictions, train_accs, test_accs  # Updated return

# Setup network skeleton 
network_input_size = 784
layer_output_sizes = [512, 256, 128, 10]  # output 10 classes
#activation_funcs = [ReLU, ReLU, softmax]
activation_ders = [ReLU_der, ReLU_der, ReLU_der, softmax_der]

#NEW
# After (dropout on hidden layers only!)
activation_funcs = [
    lambda z: ReLU_with_dropout(z, keep_prob=0.99, training=True),
    lambda z: ReLU_with_dropout(z, keep_prob=0.99, training=True),
    lambda z: ReLU_with_dropout(z, keep_prob=0.99, training=True),
    softmax
]

# Train
epochs = 50  
learning_rate = 0.05
batch_size = 100

# Create layers
layers = create_layers_batch(network_input_size, layer_output_sizes)

trained_layers, losses, train_predictions, train_accs, test_accs = train_network(
    X_train, y_train, layers, activation_funcs, activation_ders, learning_rate, epochs, batch_size
)

# Plot loss as function of epoch
plt.figure(figsize=(10, 6))
plt.plot(range(1, epochs + 1), losses, label='Loss', marker='o')
plt.xlabel('Epoch')
plt.ylabel('Cross-Entropy Loss')
plt.title('Loss vs Epoch')
plt.legend()
plt.grid(True)
plt.show()

# Plot accuracies (train and test) as function of epoch
plt.figure(figsize=(10, 6))
plt.plot(range(1, epochs + 1), train_accs, label='Train Accuracy', marker='o', linestyle='-')
plt.plot(range(1, epochs + 1), test_accs, label='Test Accuracy', marker='x', linestyle='--')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Train and Test Accuracy vs Epoch')
plt.legend()
plt.grid(True)
plt.show()
"""
#Scikit-learn Logistic Regression comparison
model = LogisticRegression(solver='saga', multi_class='multinomial', max_iter=10, random_state=42)
model.fit(X_train, y_train_labels)
y_pred = model.predict(X_test)
sk_accuracy = accuracy_score(y_test_labels, y_pred)
print(f"Scikit-learn Model Accuracy: {sk_accuracy:.4f}")
"""

NameError: name 'y_train' is not defined

In [ ]:
"""
learning_rate = 0.02
Epoch 148, Loss: 0.149, Test Accuracy: 0.930, Train Accuracy: 0.956
Epoch 149, Loss: 0.155, Test Accuracy: 0.928, Train Accuracy: 0.954
Epoch 150, Loss: 0.151, Test Accuracy: 0.927, Train Accuracy: 0.956

Discuss your results and give a critical analysis of the various parameters, including hyper-parameters like the learning rates and the regularization parameter 
, various activation functions, number of hidden layers and nodes and activation functions
"""



'\nlearning_rate = 0.02\nEpoch 148, Loss: 0.149, Test Accuracy: 0.930, Train Accuracy: 0.956\nEpoch 149, Loss: 0.155, Test Accuracy: 0.928, Train Accuracy: 0.954\nEpoch 150, Loss: 0.151, Test Accuracy: 0.927, Train Accuracy: 0.956\n\nDiscuss your results and give a critical analysis of the various parameters, including hyper-parameters like the learning rates and the regularization parameter \n, various activation functions, number of hidden layers and nodes and activation functions\n'

In [ ]:
"""
# (W, b)  new params
# (W_g, b_g) gradients of old params 



 layers_grad = backpropagation_batch(input_batch, layers, activation_funcs, target_batch, activation_ders, cost_der=cross_entropy_der)
            for (W, b), (W_g, b_g) in zip(layers, layers_grad):
                W -= learning_rate * W_g
                b -= learning_rate * b_g



 layers_grad = backpropagation_batch(input_batch, layers, activation_funcs, target_batch, activation_ders, cost_der=cross_entropy_der)
            for (W, b), (W_g, b_g) in zip(layers, layers_grad):
                W -= ADAM(W_g)
                b -= ADAM(b_g)


def ADAM(gradient):
   
   beta1 = 0.9            #dacay rate nr 1
   beta2 = 0.999          #decay rate nr 2
   alpha = 0.001          #learning rate  
   epsilon = 10**-8 

   #init values
   m_tprev = 0         #m for previous time step 
   v_tprev = 0
   t = 1               #current t, starting counting at 1    
   
   m_t = beta1 * (m_tprev) + (1-beta1) * gradient
   v_t = beta2 * v_tprev + (1-beta2)*(gradient**2)

   m_hat = m_t / (1-beta1**t)
   v_hat = v_t / (1-beta2**t)

   return alpha * m_hat/(np.sqrt(v_hat)+epsilon)

   



#normal GD
W -= learning_rate * W_g
"""

'\n# (W, b)  new params\n# (W_g, b_g) gradients of old params \n\n\n\n layers_grad = backpropagation_batch(input_batch, layers, activation_funcs, target_batch, activation_ders, cost_der=cross_entropy_der)\n            for (W, b), (W_g, b_g) in zip(layers, layers_grad):\n                W -= learning_rate * W_g\n                b -= learning_rate * b_g\n\n\n\n layers_grad = backpropagation_batch(input_batch, layers, activation_funcs, target_batch, activation_ders, cost_der=cross_entropy_der)\n            for (W, b), (W_g, b_g) in zip(layers, layers_grad):\n                W -= ADAM(W_g)\n                b -= ADAM(b_g)\n\n\ndef ADAM(gradient):\n   \n   beta1 = 0.9            #dacay rate nr 1\n   beta2 = 0.999          #decay rate nr 2\n   alpha = 0.001          #learning rate  \n   epsilon = 10**-8 \n\n   #init values\n   m_tprev = 0         #m for previous time step \n   v_tprev = 0\n   t = 1               #current t, starting counting at 1    \n   \n   m_t = beta1 * (m_tprev) + (1-beta